<a href="https://colab.research.google.com/github/ShefaRahatAli/Brainwave-Matrix-Solution/blob/main/FakeNewsDetector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## ***Fake News Detector Model***

In [20]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [18]:
import numpy as np
import re
import pandas as pd

from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [6]:
news_df = pd.read_csv('train.csv')

In [7]:
news_df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


## About the Dataset:




*  id: unique id for a news article
*  title: the title of a news article
*   author: author of the news article
*   text: the text of the article; could be incomplete
*   label: a label that marks whether the news article is real or fake:
1. 1: Fake news
2. 0: real News




## 1 Preprocessing

In [8]:
news_df.isnull().sum()

,0
id,0
title,558
author,1957
text,39
label,0


In [ ]:
news_df.shape

(20800, 5)

In [9]:
news_df = news_df.fillna(' ')

In [10]:
news_df.isnull().sum()

,0
id,0
title,0
author,0
text,0
label,0


In [11]:
news_df['content'] = news_df['author']+' '+news_df['title']

In [12]:
news_df

,id,title,author,text,label,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,Darrell Lucus House Dem Aide: We Didn’t Even S...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,"Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,Consortiumnews.com Why the Truth Might Get You...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,Jessica Purkiss 15 Civilians Killed In Single ...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,Howard Portnoy Iranian woman jailed for fictio...
...,...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0,Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0,"Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma..."
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0,Michael J. de la Merced and Rachel Abrams Macy...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1,"Alex Ansary NATO, Russia To Hold Parallel Exer..."


## separating the data & label

In [13]:
X = news_df.drop('label',axis=1)
y = news_df['label']

In [14]:
print(X)

          id                                              title  \
0          0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1          1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2          2                  Why the Truth Might Get You Fired   
3          3  15 Civilians Killed In Single US Airstrike Hav...   
4          4  Iranian woman jailed for fictional unpublished...   
...      ...                                                ...   
20795  20795  Rapper T.I.: Trump a ’Poster Child For White S...   
20796  20796  N.F.L. Playoffs: Schedule, Matchups and Odds -...   
20797  20797  Macy’s Is Said to Receive Takeover Approach by...   
20798  20798  NATO, Russia To Hold Parallel Exercises In Bal...   
20799  20799                          What Keeps the F-35 Alive   

                                          author  \
0                                  Darrell Lucus   
1                                Daniel J. Flynn   
2                             Consortiu

## Stemming:



*   Stemming is the process of reducing a word to its Root word

*  example: hung hanged hanging ======hang



## Steps:



*   lower case
*   splitting
*   removing stopwords
*   stemming





In [16]:
ps = PorterStemmer()
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]',' ',content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [ps.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

In [21]:
news_df['content'] = news_df['content'].apply(stemming)

In [22]:
news_df['content']

,content
0,darrel lucu hou dem aid even see comey letter ...
1,daniel j flynn flynn hillari clinton big woman...
2,consortiumnew com truth might get fire
3,jessica purkiss civilian kill singl us airstri...
4,howard portnoy iranian woman jail fiction unpu...
...,...
20795,jerom hudson rapper trump poster child white s...
20796,benjamin hoffman n f l playoff schedul matchup...
20797,michael j de la merc rachel abram maci said re...
20798,alex ansari nato russia hold parallel exerci b...


## separating the data and label

In [23]:
X = news_df['content'].values
y = news_df['label'].values

## converting the textual data to numerical data

In [24]:
vector = TfidfVectorizer()
vector.fit(X)
X = vector.transform(X)

In [27]:
print(X)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 210584 stored elements and shape (20800, 16984)>
  Coords	Values
  (0, 263)	0.2701012497770876
  (0, 2464)	0.36765196867972083
  (0, 2933)	0.24684501285337127
  (0, 3567)	0.3598939188262558
  (0, 3759)	0.27053324808454915
  (0, 4917)	0.23331696690935097
  (0, 6930)	0.2187416908935914
  (0, 7612)	0.24785219520671598
  (0, 8546)	0.2921251408704368
  (0, 8822)	0.36359638063260746
  (0, 13348)	0.2565896679337956
  (0, 15553)	0.2848506356272864
  (1, 1481)	0.2957471154505952
  (1, 1877)	0.15614790568229528
  (1, 2206)	0.36915639258038363
  (1, 2790)	0.19208753385709676
  (1, 3535)	0.2653147533915268
  (1, 5440)	0.7186013955384664
  (1, 6744)	0.19152496072048605
  (1, 16656)	0.3025156488372128
  (2, 2917)	0.3179886800654691
  (2, 3072)	0.46097489583229645
  (2, 5326)	0.3866530551182615
  (2, 5903)	0.3474613386728292
  (2, 9532)	0.49351492943649944
  :	:
  (20797, 3610)	0.211655450844435
  (20797, 6969)	0.21809398920480086
  (20797

## Splitting the dataset to training & test data

In [28]:
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size = 0.2, stratify=y, random_state=2)

In [29]:
X_train.shape

(16640, 16984)

## Training the Model: Logistic Regression

In [30]:
model = LogisticRegression()
model.fit(X_train,Y_train)

LogisticRegression()

**In a Jupyter environment, please rerun this cell to show the HTML representation or trust the notebook.
On GitHub, the HTML representation is unable to render, please try loading this page with nbviewer.org.**

In [31]:
# on training set
train_y_pred = model.predict(X_train)
print(accuracy_score(train_y_pred,Y_train))

0.9863581730769231


In [32]:
# on testing set
testing_y_pred = model.predict(X_test)
print(accuracy_score(testing_y_pred,Y_test))

0.9790865384615385


## Detection System

In [33]:
input_data = X_test[10]
prediction = model.predict(input_data)

In [34]:
if prediction[0] == 0:
    print('The News Is Real')
else:
    print('The News is Fake')

The News Is Real


In [35]:
news_df['content'][2]

'consortiumnew com truth might get fire'

# ***Fake News Detector***

In [36]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords # the for of in with
from nltk.stem.porter import PorterStemmer # loved loving == love
from sklearn.feature_extraction.text import TfidfVectorizer # loved = [0.0]
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [37]:
news_df = pd.read_csv('train.csv')

In [38]:
news_df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [40]:
news_df.shape

(20800, 5)

In [41]:
news_df.isna().sum()

,0
id,0
title,558
author,1957
text,39
label,0


In [42]:
news_df = news_df.fillna(' ')

In [43]:
news_df.isna().sum()

,0
id,0
title,0
author,0
text,0
label,0


In [44]:
news_df['content'] = news_df['author']+" "+news_df['title']

In [45]:
news_df

,id,title,author,text,label,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,Darrell Lucus House Dem Aide: We Didn’t Even S...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,"Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,Consortiumnews.com Why the Truth Might Get You...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,Jessica Purkiss 15 Civilians Killed In Single ...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,Howard Portnoy Iranian woman jailed for fictio...
...,...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0,Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0,"Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma..."
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0,Michael J. de la Merced and Rachel Abrams Macy...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1,"Alex Ansary NATO, Russia To Hold Parallel Exer..."


In [46]:
news_df['content']

,content
0,Darrell Lucus House Dem Aide: We Didn’t Even S...
1,"Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo..."
2,Consortiumnews.com Why the Truth Might Get You...
3,Jessica Purkiss 15 Civilians Killed In Single ...
4,Howard Portnoy Iranian woman jailed for fictio...
...,...
20795,Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796,"Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma..."
20797,Michael J. de la Merced and Rachel Abrams Macy...
20798,"Alex Ansary NATO, Russia To Hold Parallel Exer..."


In [47]:
# stemming
ps = PorterStemmer()
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]',' ',content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [ps.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

In [48]:
news_df['content'] = news_df['content'].apply(stemming)

In [49]:
news_df['content']

,content
0,darrel lucu hous dem aid even see comey letter...
1,daniel j flynn flynn hillari clinton big woman...
2,consortiumnew com truth might get fire
3,jessica purkiss civilian kill singl us airstri...
4,howard portnoy iranian woman jail fiction unpu...
...,...
20795,jerom hudson rapper trump poster child white s...
20796,benjamin hoffman n f l playoff schedul matchup...
20797,michael j de la merc rachel abram maci said re...
20798,alex ansari nato russia hold parallel exercis ...


In [51]:
X = news_df['content'].values
y = news_df['label'].values

In [52]:
print(X)

['darrel lucu hous dem aid even see comey letter jason chaffetz tweet'
 'daniel j flynn flynn hillari clinton big woman campu breitbart'
 'consortiumnew com truth might get fire' ...
 'michael j de la merc rachel abram maci said receiv takeov approach hudson bay new york time'
 'alex ansari nato russia hold parallel exercis balkan'
 'david swanson keep f aliv']


In [53]:
vector = TfidfVectorizer()
vector.fit(X)
X = vector.transform(X)

In [54]:
print(X)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 210687 stored elements and shape (20800, 17128)>
  Coords	Values
  (0, 267)	0.2701012497770876
  (0, 2483)	0.36765196867972083
  (0, 2959)	0.24684501285337127
  (0, 3600)	0.3598939188262558
  (0, 3792)	0.27053324808454915
  (0, 4973)	0.23331696690935097
  (0, 7005)	0.2187416908935914
  (0, 7692)	0.24785219520671598
  (0, 8630)	0.2921251408704368
  (0, 8909)	0.36359638063260746
  (0, 13473)	0.2565896679337956
  (0, 15686)	0.2848506356272864
  (1, 1497)	0.2939891562094648
  (1, 1894)	0.15521974226349364
  (1, 2223)	0.3827320386859759
  (1, 2813)	0.19094574062359204
  (1, 3568)	0.26373768806048464
  (1, 5503)	0.7143299355715573
  (1, 6816)	0.1904660198296849
  (1, 16799)	0.30071745655510157
  (2, 2943)	0.3179886800654691
  (2, 3103)	0.46097489583229645
  (2, 5389)	0.3866530551182615
  (2, 5968)	0.3474613386728292
  (2, 9620)	0.49351492943649944
  :	:
  (20797, 3643)	0.2115550061362374
  (20797, 7042)	0.21799048897828685
  (2079

In [55]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,stratify=y, random_state=1)

In [56]:
X_train.shape

(16640, 17128)

In [57]:
X_test.shape

(4160, 17128)

In [58]:
model = LogisticRegression()
model.fit(X_train,y_train)

LogisticRegression()

**In a Jupyter environment, please rerun this cell to show the HTML representation or trust the notebook.
On GitHub, the HTML representation is unable to render, please try loading this page with nbviewer.org.**

In [59]:
train_y_pred = model.predict(X_train)
print("train accurracy :",accuracy_score(train_y_pred,y_train))

train accurracy : 0.9868389423076923


In [60]:
test_y_pred = model.predict(X_test)
print("train accurracy :",accuracy_score(test_y_pred,y_test))

train accurracy : 0.9764423076923077


In [61]:
# prediction system

input_data = X_test[20]
prediction = model.predict(input_data)
if prediction[0] == 1:
    print('Fake news')
else:
    print('Real news')

Fake news


In [62]:
news_df['content'][20]

'news hope gop nude paul ryan emerg ayahuasca tent vision new republican parti'

## ***Fake-News-Detection-Machine-Learning-Scam-Detection-NLP***

In [65]:
import pandas as pd




In [66]:
df = pd.read_csv("train.csv")

In [67]:
df = df[['text','label']]

In [68]:
df.head()

,text,label
0,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,Ever get the feeling your life circles the rou...,0
2,"Why the Truth Might Get You Fired October 29, ...",1
3,Videos 15 Civilians Killed In Single US Airstr...,1
4,Print \nAn Iranian woman has been sentenced to...,1


In [69]:
df = df.sample(n=7000)
df.shape

(7000, 2)

In [70]:
df.isnull().sum()

,0
text,14
label,0


In [71]:
df.dropna(inplace=True)

## Clean Text

In [76]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [77]:
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

def clean_text(text):
    # Convert text to lowercase
    text = text.lower()

    # Remove special characters and digits
    text = re.sub(r'[^a-zA-Z\s]', '', text)

    # Remove links
    text = re.sub(r'http\S+', '', text)

    # Tokenize the text
    words = word_tokenize(text)

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    filtered_words = [word for word in words if word not in stop_words]

    # Initialize Porter Stemmer
    stemmer = PorterStemmer()

    # Perform stemming
    stemmed_words = [stemmer.stem(word) for word in filtered_words]

    # Join the stemmed words back into a single string
    cleaned_text = ' '.join(stemmed_words)

    return cleaned_text

In [78]:
df['clean_text'] = df['text'].apply(lambda x: clean_text(x))

In [79]:
df['label'].value_counts()

,count
label,
1,3528
0,3458


## ***Train Models***

In [80]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report


# Train-test split
X_train, X_test, y_train, y_test = train_test_split(df['clean_text'], df['label'], test_size=0.2, random_state=42)

# TF-IDF vectorization
tfidf_vectorizer = TfidfVectorizer()
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

In [81]:
# Random Forest classifier
rf_classifier = RandomForestClassifier(random_state=42)
rf_classifier.fit(X_train_tfidf, y_train)

# Predictions
y_pred = rf_classifier.predict(X_test_tfidf)

# Confusion matrix and classification report
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

print("Confusion Matrix:")
print(conf_matrix)
print("\nClassification Report:")
print(class_report)

Confusion Matrix:
[[641  47]
 [103 607]]

Classification Report:
              precision    recall  f1-score   support

           0       0.86      0.93      0.90       688
           1       0.93      0.85      0.89       710

    accuracy                           0.89      1398
   macro avg       0.89      0.89      0.89      1398
weighted avg       0.90      0.89      0.89      1398



In [82]:
from sklearn.linear_model import LogisticRegression

# Logistic Regression classifier
lr_classifier = LogisticRegression(random_state=42)
lr_classifier.fit(X_train_tfidf, y_train)

# Predictions
y_pred_lr = lr_classifier.predict(X_test_tfidf)

# Confusion matrix and classification report for logistic regression
conf_matrix_lr = confusion_matrix(y_test, y_pred_lr)
class_report_lr = classification_report(y_test, y_pred_lr)

print("Logistic Regression - Confusion Matrix:")
print(conf_matrix_lr)
print("\nLogistic Regression - Classification Report:")
print(class_report_lr)

Logistic Regression - Confusion Matrix:
[[625  63]
 [ 34 676]]

Logistic Regression - Classification Report:
              precision    recall  f1-score   support

           0       0.95      0.91      0.93       688
           1       0.91      0.95      0.93       710

    accuracy                           0.93      1398
   macro avg       0.93      0.93      0.93      1398
weighted avg       0.93      0.93      0.93      1398



In [83]:
from sklearn.svm import SVC

# SVM classifier
svm_classifier = SVC(kernel='linear', random_state=42)
svm_classifier.fit(X_train_tfidf, y_train)

# Predictions
y_pred_svm = svm_classifier.predict(X_test_tfidf)

# Confusion matrix and classification report for SVM
conf_matrix_svm = confusion_matrix(y_test, y_pred_svm)
class_report_svm = classification_report(y_test, y_pred_svm)

print("SVM - Confusion Matrix:")
print(conf_matrix_svm)
print("\nSVM - Classification Report:")
print(class_report_svm)

SVM - Confusion Matrix:
[[638  50]
 [ 30 680]]

SVM - Classification Report:
              precision    recall  f1-score   support

           0       0.96      0.93      0.94       688
           1       0.93      0.96      0.94       710

    accuracy                           0.94      1398
   macro avg       0.94      0.94      0.94      1398
weighted avg       0.94      0.94      0.94      1398



## ***Prediction System***

In [84]:
def predict_fake_or_real(text):
    # Clean the input text
    cleaned_text = clean_text(text)

    # Transform the cleaned text using the TF-IDF vectorizer
    text_tfidf = tfidf_vectorizer.transform([cleaned_text])

    # Use the trained classifier to predict
    prediction = svm_classifier.predict(text_tfidf)

    # Map prediction to label
    label = "fake" if prediction[0] == 1 else "real"

    return label

# Example usage:
input_text = 'Organizing for Action, the activist group that morphed from Barack Obama’s first presidential campaign, has partnered with the   Indivisible Project for “online trainings” on how to protest President Donald Trump’s agenda. [Last week, Breitbart News extensively reported that Indivisible leaders are openly associated with groups financed by billionaire George Soros.  Politico earlier this month profiled Indivisible in an article titled, “Inside the protest movement that has Republicans reeling. ”  The news agency not only left out the Soros links, but failed to note that the organizations cited in its article as helping to amplify Indivisible’s message are either financed directly by Soros or have close ties to groups funded by the billionaire, as Breitbart News documented. Organizing for Action (OFA) is a   community organizing project that sprung from Obama’s 2012 campaign organization, Organizing for America, becoming a nonprofit described by the Washington Post as “advocate[ing] for the president’s policies. ” In a recent Facebook post titled, “Take a deep breath. Then take action,” OFA called on constituents to lobby particularly hard between now and February 26, when lawmakers will be in their home districts. The post included a link to a guide released by Indivisible on how to organize against Trump. “Stay tuned for online trainings and invitations to calls with coalition partners like Indivisible Guide,” the OFA post states. Paul Sperry, writing at the New York Post, relates: The manual, published with OFA partner “Indivisible,” advises protesters to go into halls quietly so as not to raise alarms, and “grab seats at the front of the room but do not all sit together. ” Rather, spread out in pairs to make it seem like the whole room opposes the Republican host’s positions. “This will help reinforce the impression of broad consensus. ” It also urges them to ask “hostile” questions  —   while keeping “a firm hold on the mic”  —   and loudly boo the the GOP politician if he isn’t “giving you real answers. ” “Express your concern [to the event’s hosts] they are giving a platform to   authoritarianism, racism, and corruption,” it says.   …    “Even the safest [Republican] will be deeply alarmed by signs of organized opposition,” the document states, “because these actions create the impression that they’re not connected to their district and not listening to their constituents. ” Sperry reported OFA “plans to stage 400 rallies across 42 states this year to attack Trump and Republicans over ObamaCare’s repeal. ” Earlier this month, NBC News reported on OFA’s new actions and its partnership with Indivisible: OFA has hired 14 field organizers in states home to key senators as part of its campaign to defend Obama’s signature healthcare law. To run that campaign, the group hired Saumya Narechania  —   the former national field director at Enroll America, which worked to sign people up for Obamacare  —   and a deputy campaign manager.   …    OFA says more than 1, 800 people have applied to its Spring Community Engagement Fellowship, a   training program,   of whom have not previously been involved with OFA. And the group has teamed up with Indivisible, a buzzy newcomer to the progressive movement, to offer organizing training that began Thursday night with a video conference. A combined 25, 000 people have registered to participate in those trainings, OFA said. Indivisible’s DC branch was implicated in a scuffle last week that reportedly injured a    staffer for Rep. Dana Rohrabacher ( ) as well as reportedly knocking a    to the ground.  Protesters claimed they were only delivering Valentine’s Day cards. Indivisible is a part of a coalition of activist groups slated to hold a massive   Tax March in Washington and at least 60 other locations on April 15. Unreported by the news media is that most of the listed partners and support organizers of the march are openly financed by Soros or have close links to Soros financing, as Breitbart News documented last week. Meanwhile, earlier this month, Politico profiled Indivisible and reported that “conservatives” are “spreading unfounded rumors” that the group is “being driven by wealthy donors like George Soros. ” Politico, however, seemingly failed to do even the most minimal research on the Indivisible leaders cited in the news outlet’s own profile.  Some of those personalities are openly associated with groups financed by Soros. Politico further failed to note that the organizations cited in its article as helping to amplify Indivisible’s message are either financed directly by Soros or have close ties to groups funded by the billionaire. Citing Angel Padilla, a   of the group, Politico reported: Dubbed “Indivisible,” the group launched as a way for Padilla and a handful of fellow   aides to channel their   heartbreak into a manual for quashing President Donald Trump’s agenda. They drafted a   protest guide for activists, full of pointers on how to bird dog their members of Congress in the language of Capitol insiders. The manual has since been downloaded over one million times. Indivisible says on its website that over 4, 500 local groups across the nation have “signed up to resist the Trump agenda in nearly every congressional district in the country. ” The manual has been utilized to form the basis of a protest movement. The group’s website states: “What’s more, you all are putting the guide into action —  showing up en masse to congressional district offices and events, and flooding the congressional phone lines. You’re resisting —  and it’s working. ” Politico reported on “unfounded” rumors being spread about Soros’s involvement with Indivisible (emphasis added by this reporter): Its handful of senior leaders count about 100 contributors to their national organizing work but insist that all are working on a volunteer basis. They know conservatives are spreading unfounded rumors that their success is being driven by wealthy donors like George Soros, which they flatly deny. That paragraph was followed by the following quote from   Padilla (emphasis again added by this reporter): “It doesn’t matter who we take money from  —   we’re always going to get blamed as a Soros group, even if we don’t take money from Soros,” said Padilla, now an analyst with the National Immigration Law Center. “That’s one of the attacks and that’s fine. ” While “Indivisible” has yet to disclose its donors, Politico failed to inform readers that the National Immigration Law Center where the news outlet reported Padilla serves as an analyst is financed by Soros’s Open Society Foundations. The Center has received numerous Open Society grants earmarked for general support. Also unmentioned by Politico is that Padilla previously served as an immigration policy consultant at the radical National Council of La Raza. Soros is a major La Raza donor. Politico went on to detail how Indivisible has been aided by MoveOn. org and the ACLU.  The news website failed to tell readers that MoveOn. org and the ACLU are both financed by Soros, a relevant tidbit given Politico’s claim about “unfounded rumors” that Indivisibles’ success was being driven by Soros .  The news website reported: In addition, MoveOn. org and the Working Families Party joined with Indivisible for its first nationwide call on Jan. 22. Nearly 60, 000 people phoned in that day, according to Levin and MoveOn organizing director Victoria Kaplan. Indivisible estimates that its second national call, on the impact of Trump’s immigration order with assistance from the ACLU and Padilla’s group, drew 35, 000 people. Politico also missed that, according to its Twitter account, another organizer of the conference call with MoveOn. org was the International Refugee Assistance Project, a project of the Urban Justice Center, another recipient of an Open Society grant. Taryn Higashi, executive director of the Center’s International Refugee Assistance Project, currently serves on the Advisory Board of the International Migration Initiative of Soros’s Open Society Foundations. Politico further reported on Indivisible’s ties to the organizers of last month’s   Women’s March while failing to mention that Soros reportedly has ties to more than 50 “partners” of that march. Also, this journalist first reported on the march leaders’ own close associations with Soros. Regarding Indivisible and the Women’s March, Politico reported: Indivisible is also embracing collaboration with other major   protest outlets. Leaders of the group were in communication with Women’s March organizers before their main event on Jan. 21, and that partnership will become official when the March unveils the third in its series of 10 direct actions that attendees have been asked to pursue in their communities. Another Indivisible leader mentioned in the Politico article is Jeremy Haile. Not reported by Politico is that is Haile served as federal advocacy counsel for the Sentencing Project.  The Sentencing Project is reportedly financed by Soros’s Open Society Foundations, which has also hosted the Project to promote its cause. Aaron Klein is Breitbart’s Jerusalem bureau chief and senior investigative reporter. He is a New York Times bestselling author and hosts the popular weekend talk radio program, “Aaron Klein Investigative Radio. ” Follow him on Twitter @AaronKleinShow. Follow him on Facebook'
prediction = predict_fake_or_real(input_text)
print("Prediction:", prediction)

Prediction: real


In [85]:
input_text = 'Email \nThe alleged connection between Donald Trump and Russia — asserted by Hillary Clinton in both the second and third presidential debates as a dodge for the leaked e-mails and other documents which have hounded her candidacy — has dominated much of the speculative attention of both the mainstream media and social media for weeks. Is there anything to it? Is Trump, in essence, Putin’s puppet, as Clinton claimed in the third debate? \nIn an interesting twist of logic (if one can use that word to describe the incoherent and dissonant arguments made by a woman who still denies — despite mountains of evidence to the contrary — that she sent and received classified data over her unsecured, private e-mail server), Hillary Clinton at once claims that Russia is responsible for the hack of the servers of the Democratic National Committee (DNC) in late spring and that there is no way her server was hacked by anyone. Leveraging the anti-Russian sentiment to its fullest, she and others in the DNC have sought to shift the focus away from what was on the servers and toward blaming Russia for the hack. After all, if Russia is trying to throw the election to Trump, wouldn’t the patriotic vote be for Clinton? \nBy the time WikiLeaks announced that a trove of damning documents and e-mails would be published on the Internet, Clinton and the DNC (along with their lapdog media) were already in full-blown "it was Russia" mode. On the same day that a video of Donald Trump emerged in which he can be heard bragging of his sexual abuse of women — which he has since claimed was just “locker room talk” — WikiLeaks began a rolling release of the promised leaked documents. Both of these stories deserved media attention . Only one got it. \nBy the time the media was ready to report anything on the WikiLeaks dump, it was to speculate as to what part Russia played in the initial hack. Clinton — in her role as the staunch anti-Russian — used the timing of the second debate to attack Trump for the video and his supposed ties to Russia, which she plainly blamed for the leaked documents, saying to moderator Martha Raddatz: But, you know, let’s talk about what’s really going on here, Martha, because our intelligence community just came out and said in the last few days that the Kremlin, meaning Putin and the Russian government, are directing the attacks, the hacking on American accounts to influence our election. And WikiLeaks is part of that, as are other sites where the Russians hack information, we don’t even know if it’s accurate information, and then they put it out. We have never in the history of our country been in a situation where an adversary, a foreign power, is working so hard to influence the outcome of the election. And believe me, they’re not doing it to get me elected. They’re doing it to try to influence the election for Donald Trump. \nClinton went even further in the third debate , bringing up Putin in an attempt to dodge a question about a leaked document showing that she gave a speech to a foreign bank in which she said, “My dream is a hemispheric common market with open trade and open borders.” She — again — put the emphasis on the fact that WikiLeaks was providing the leaked documents rather than on what the leaked documents have to say. Then she claimed, on the authority of “17 of our intelligence agencies,” that Russia was behind the hacked and leaked documents: But you are very clearly quoting from WikiLeaks. And what’s really important about WikiLeaks is that the Russian government has engaged in espionage against Americans. They have hacked American websites, American accounts of private people, of institutions. Then they have given that information to WikiLeaks for the purpose of putting it on the Internet. This has come from the highest levels of the Russian government, clearly, from Putin himself, in an effort, as 17 of our intelligence agencies have confirmed, to influence our election. \nTrump rebutted by bringing the conversation back on topic (not his usual strongest point), saying: She wants open borders. People are going to pour into our country. People are going to come in from Syria. She wants 550 percent more people than Barack Obama, and he has thousands and thousands of people. They have no idea where they come from. And you see, we are going to stop radical Islamic terrorism in this country. She won’t even mention the words, and neither will President Obama. So I just want to tell you, she wants open borders. Now we can talk about Putin. I don’t know Putin. He said nice things about me. If we got along well, that would be good. If Russia and the United States got along well and went after ISIS, that would be good. He has no respect for her. He has no respect for our president. And I’ll tell you what: We’re in very serious trouble, because we have a country with tremendous numbers of nuclear warheads — 1,800, by the way — where they expanded and we didn’t, 1,800 nuclear warheads. And she’s playing chicken. Look, Putin... \nAt this point, Clinton interrupted to say, “Well, that’s because he’d rather have a puppet as president of the United States.” \nSo, getting past the he-said-she-said, what are the facts as we know them? \nOn Monday, the New York Times reported that after conducting a months-long “investigation into a Russian role in the American presidential campaign” in which “agents scrutinized advisers close to Donald J. Trump, looked for financial connections with Russian financial figures, searched for those involved in hacking the computers of Democrats, and even chased a lead — which they ultimately came to doubt — about a possible secret channel of email communication from the Trump Organization to a Russian bank,” the FBI “sees no clear link” between Trump and Russia. Of course, in typical leftist fashion, the Times not-so-gently insinuates that the FBI is guilty of pulling punches in the investigation. Interestingly, the Times seemed to miss that that is exactly what happened in the FBI\'s investigation into Clinton\'s e-mail server. \nAs for Clinton’s claims that the purpose of the hacked and leaked documents and e-mails was to influence the election for Trump, the Times article admits: Law enforcement officials say that none of the investigations so far have found any conclusive or direct link between Mr. Trump and the Russian government. And even the hacking into Democratic emails, F.B.I. and intelligence officials now believe, was aimed at disrupting the presidential election rather than electing Mr. Trump. \nAlso on Monday, Slate published an article claiming that a server belonging to Donald Trump was “communicating in a secretive fashion” with servers in Russia. By the next day, the Washington Post had debunked the Slate article, saying, “That secret Trump-Russia email server link is likely neither secret nor a Trump-Russia link.” Based in part on an interview with Naadir Jeewa, who “does consulting work on precisely the sorts of systems involved in” the scenario involving the server Slate claims was acting as a conduit between Trump and Russia, the article by the Post explains: To understand what\'s likely happening, we need to establish a few basics. First of all, the Trump server wasn\'t really a Trump server. It was much less of a Trump email server, for example, than Hillary Clinton\'s email server was hers. Clinton had a physical server that hosted her email. The trump-email.com domain that Alfa was connecting to was hosted by a company called Cendyn. Cendyn runs marketing systems for the hospitality industry, meaning that it offers an out-of-the-box solution for a company that owns a bunch of hotels to push out sales pitch emails to its customers. In other words, trump-email.com isn\'t the email server Trump used to send emails from his closet. It was a domain name that linked back to a Cendyn server. This is important for a few reasons. The first, Jeewa said, was that the trump-email.com was configured to reject a certain type of query from another server. Since its job was simply to push out thousands of enticements to come stay at Trump Soho (or whatever) it didn\'t need to receive many incoming requests (like incoming email). The second is that the conspiracy theory hinges on Trump\'s team using an offsite server hosted by someone else for its quiet communications with its Russian allies. Instead of, say, their own server, under their own control. Or an encrypted chat app. Or a phone call. \nThis writer has to admit to being amused to see the shoe on the other foot (or the tinfoil hat on the other head, in this instance) as the Left trots out wild conspiracy theories to create a Trump-Putin connection to make Clinton retroactively correct. In point of fact, Clinton might be better off trying to implicate Trump in the Kennedy assassination. \nBut, what about Clinton’s assertion that WikiLeaks is releasing documents pilfered by Russia for the sake of influencing the election for Trump? Aside from the FBI saying that there does not appear to be any attempt to influence the election for Trump, Julian Assange, the founder and public face of WikiLeaks, denies that Russian hackers are his source. \nIn an upcoming documentary made by Dartmouth Films, Assange speaks of the “Clinton camp” putting forth a “hysteria that Russia is responsible for everything.” He goes on to say, “Hillary Clinton has stated multiple times, falsely, that 17 US intelligence agencies had assessed that Russia was the source of our publications. That’s false — we can say that the Russian government is not the source.” \nFurthermore, the Kremlin denies Clinton’s claims. Russian presidential press secretary Dmitry Peskov called the claims “nonsense.” While that — in and of itself — would not carry much weight, when it is added to the body of evidence that includes Assange denying the Russian connection, the laughably far-fetched lengths to which the Left will got to promote easily debunkable conspiracy theories, the FBI stating that there is “no clear link” between Trump and Russia, and Trump himself denying that he has any dealings with Russia, it’s fairly easy to see that the only thing there is to see here is an imploding campaign by the woman who — the last time she occupied 1600 Pennsylvania Avenue — claimed that her husband’s impeachment was the result of a “vast right-wing conspiracy.” \nSome things never change. Photos: AP Images  '
prediction = predict_fake_or_real(input_text)
print("Prediction:", prediction)

Prediction: fake
